# FINAL DATA MINING 

## Khai báo

### Import các packages cần thiết 

In [35]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import numpy
import seaborn as sns
import time
import copy
from tqdm import tqdm
from time import sleep
from selenium.common.exceptions import NoSuchElementException



### Định nghĩa hàm Crawler

In [36]:
class Crawler:
    def __init__(self, url):
        # Declare browser
        self.driver = webdriver.Chrome(r'./chromedriver')
        self.driver.maximize_window()
        #self.driver = webdriver.Edge('./msedgedriver.exe')
        #self.driver = webdriver.Chrome()
        #self.driver = webdriver.Firefox()
        #self.driver = webdriver.Safari()
        self.url = url
        
    def startCrawling(self):
        self.driver.get(self.url)

    def clickBtn(self, xpath):
        btn = self.driver.find_element(by=By.XPATH, value=xpath)
        self.driver.execute_script("arguments[0].click();", btn)
        
    def enterInputDate(self, xpath, message):
        inputTag = self.driver.find_element(by=By.XPATH, value=xpath)
        
        inputTag.clear()
        inputTag.send_keys(message)

    def extractSymbol(self, num):
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        webContent = self.soup.find("table", {"id": "tblScroll1"}).find_all("td", {"class" : "StockSymbolEven"})

        data = []        
        for i in webContent:
            data.append(i.text)

        data = numpy.random.choice(data, num, replace = False)
        return data
    
    def extractData(self):
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        
        webContent = self.soup.find("table", {"id": "GirdTable2"}).find_all("tr")
        priceClosingData = []
        for i in webContent:
            priceClosingColumn = str(i.find_all("td")[2].text).replace(u'\xa0', u'')
            dateColumn = i.find("td", {"class": "Item_DateItem"})
                        
            if priceClosingColumn != "KL" and dateColumn != None:
                priceClosingData.append((dateColumn.text, priceClosingColumn))
            
        return priceClosingData
    

### Khai báo hàm xử  lý File CSV

In [37]:
class CSVHandler:
    def __init__(self):
        self.df = {}
        
    def createDataFrame(self, stockSymbolData):
        keys = list(stockSymbolData.keys())
        data = {}
        
        #Lấy cột ngày
        date = [i[0] for i in stockSymbolData[keys[0]]]
        date.reverse()
        
        data['Date'] = date #cột Date
        
        #Lấy các cột mã chứng khoán và giá đóng cửa
        for stockSymbol in keys:
            stockClosingPrice = [float(i[1]) for i in stockSymbolData[stockSymbol]]
            stockClosingPrice.reverse()
            
            data[stockSymbol] = stockClosingPrice ## Cột StockSymbol
        
        self.df = pd.DataFrame(data)
        
    def exportCSV(self, fileName):
        self.df.to_csv(fileName, index = False, header=True)
        
    def readCSVToDataFrame(self, fileName):
        dataFrame = pd.read_csv(fileName)
        return dataFrame

### Khai báo các hàm xử lý data sau khi cào dữ liệu

In [38]:
#Một số hàm nhỏ dành cho tiền xử lí dữ liệu trước khi sử dụng nó

## Hàm xử lí dữ liệu của mã có ngày bị trùng
def handleDuplicateData(stockSymbolData):
    for i in stockSymbolData:
        tempList = []
        [tempList.append(j) for j in stockSymbolData[i] if j not in tempList]       
        stockSymbolData[i] = tempList
    
    return stockSymbolData

## Hàm chỉ lấy những ngày mà 100 mã đều có giá đóng cửa
def handleMissingDate(stockSymbolData):
    dateList = []
    for i in stockSymbolData:
        for j in stockSymbolData[i]:
            if j[0] not in dateList:
                dateList.append(j[0])
    
    removeDateList = []
    for i in dateList:
        for j in stockSymbolData:
            if i not in [k[0] for k in stockSymbolData[j]] and i not in removeDateList:
                removeDateList.append(i)    
    
    for i in stockSymbolData:
        count = 0
        tempList = []
        for j in stockSymbolData[i]:
            if j[0] not in removeDateList:
                tempList.append(j)
        if(len(tempList) > 0):
            stockSymbolData[i] = tempList

    return stockSymbolData

## Thực thi

### Lấy Stock Symbol

In [39]:
bot = Crawler('https://quotes.vcbs.com.vn/a/exchange.html?symbol=HSX')
bot.startCrawling()

#Lấy 200 mã
time.sleep(3)
stock_symbol = bot.extractSymbol(200)

/tmp/ipykernel_102845/2992471288.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(r'./chromedriver')


<b>Chú thích:</b> <br>
Vì có nhưng mã chứng khoán sẽ có khả năng không có (biến mất hoặc bằng một lý do nào đó) trong một khoảng thời gian nhất định. Vì vậy để đảm bảo được có thể cào được 100 mã còn nguyên vẹn(Có cả 2020, 2021, quý 1 2022), ta nên lấy dư để trừ hao.


#### Sample

In [40]:
# stock_symbol_lst = ['CLL', 'HPX', 'HVN', 'SBT', 'CTS', 'VCI', 'HAR', 'TCT', 'PGI', 'DMC', 'MIG', 'PVD', 'DHA', 'VPH', 'VOS', 'SHI', 'VNM', 'AST', 'SFI', 'FDC', 'IDI', 'VCF', 'SC5', 'LGC', 'KDH', 'CCL', 'DBT', 'BFC', 'PDR', 'CTF', 'OPC', 'DGC', 'AGM', 'LAF', 'ROS', 'PAC', 'MCG', 'TMT', 'VJC', 'PC1', 'TMP', 'CRE', 'TVS', 'ACB', 'TGG', 'SMB', 'BRC', 'CEE', 'TNT', 'SGR', 'NHH', 'ITA', 'HRC', 'GTA', 'QCG', 'HOT', 'DTL', 'DIG', 'PIT', 'TDM', 'SII', 'SPM', 'PHC', 'VGC', 'HDG', 'FMC', 'TTE', 'TV2', 'DCL', 'SRF', 'EIB', 'VSH', 'LCG', 'KSB', 'GDT', 'HAX', 'IBC', 'NKG', 'VRE', 'HAG', 'GMC', 'SJF', 'SKG', 'ITD', 'NT2', 'DBC', 'KBC', 'D2D', 'KOS', 'POW', 'TLG', 'VDS', 'HID', 'HCD', 'PMG', 'GEX', 'HMC', 'SFC', 'DSN', 'TPC']
# stock_symbol = numpy.array( stock_symbol_lst)
# print(stock_symbol)
# print(len(stock_symbol))

#### In Stock Symbol đã lấy được

In [41]:
print(stock_symbol)

['NHA' 'TCD' 'GMC' 'BCE' 'THI' 'DGC' 'CIG' 'SRF' 'VJC' 'D2D' 'HTN' 'FDC'
 'BSI' 'KBC' 'DRH' 'OCB' 'QBS' 'SSC' 'ABS' 'TDM' 'MSN' 'DAH' 'TYA' 'NVL'
 'VPH' 'NHT' 'CCI' 'SAB' 'BMC' 'SZC' 'SII' 'ASP' 'PMG' 'SVI' 'GDT' 'CKG'
 'NT2' 'SVC' 'IBC' 'HDB' 'HMC' 'CMX' 'GAB' 'DCL' 'TIX' 'HHS' 'ANV' 'LCG'
 'YBM' 'POW' 'DVP' 'DBT' 'VGC' 'NO1' 'COM' 'TCO' 'FMC' 'TLG' 'SMB' 'VHM'
 'VCA' 'CDC' 'VRE' 'AGG' 'ILB' 'TCT' 'TDW' 'TVS' 'SFC' 'BMP' 'SHA' 'TTA'
 'FRT' 'HOT' 'TMP' 'SGR' 'CTI' 'TNH' 'EVF' 'VIC' 'TPC' 'PNJ' 'PGC' 'GMH'
 'ACG' 'HDG' 'PVD' 'APG' 'HU1' 'KHP' 'TCL' 'DHG' 'HAG' 'PAC' 'DQC' 'MCP'
 'DXS' 'HTI' 'TV2' 'PLP' 'KDH' 'BWE' 'TNT' 'VNG' 'NLG' 'ACB' 'SAV' 'HPX'
 'CTF' 'DHA' 'BCM' 'VPD' 'TDG' 'HCD' 'SCR' 'DMC' 'AAA' 'FIT' 'HSL' 'SC5'
 'STG' 'AAT' 'VSH' 'HAX' 'CLL' 'LGL' 'CSV' 'VIP' 'KSB' 'VCI' 'EIB' 'BHN'
 'LAF' 'DPM' 'PGI' 'LPB' 'VDS' 'TMT' 'VNM' 'MBB' 'EMC' 'RAL' 'VND' 'ASG'
 'SKG' 'PDR' 'PTC' 'BID' 'SPM' 'TTE' 'GTA' 'PSH' 'ADG' 'MSB' 'DTL' 'AGR'
 'ITA' 'REE' 'NAF' 'TGG' 'FCM' 'TRC' 'HAR' 'UIC' 'H

### Thực thi cào dữ liệu

In [42]:
bot = Crawler("https://s.cafef.vn")


/tmp/ipykernel_102845/2992471288.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(r'./chromedriver')


In [43]:
data = {}
configs = [['1/1/2020', '31/12/2020', 13], ['1/1/2021', '31/12/2021', 13],['1/1/2022', '30/4/2022',4]]

for i in tqdm(stock_symbol, ncols = 100, desc ="Phần trăm hoàn tất"):
    try:
        i = i.replace('*', '')
        newUrl = "https://s.cafef.vn/Lich-su-giao-dich-" + i + "-1.chn"
        bot.url = newUrl
        bot.startCrawling()
        dateAndClosingPrice = []
        for y in configs: 
            dateStart = y[0]
            dateEnd = y[1]
            ## 2020
            bot.enterInputDate('//*[@id="ContentPlaceHolder1_ctl03_dpkTradeDate1_txtDatePicker"]', dateStart)
            bot.enterInputDate('//*[@id="ContentPlaceHolder1_ctl03_dpkTradeDate2_txtDatePicker"]', dateEnd)

            bot.clickBtn('//*[@id="ContentPlaceHolder1_ctl03_btSearch"]')

            time.sleep(2)
            
            
            #13 trang dữ liệu cần crawl
            numberOfPage = y[2]
            for j in range(0, numberOfPage):
                time.sleep(2)
                dateAndClosingPrice += bot.extractData()

                if(j == numberOfPage - 1):
                    time.sleep(1)
                    continue

                if(j == 0):
                    bot.clickBtn('//*[@id="ContentPlaceHolder1_ctl03_divHO"]/div/div/table/tbody/tr/td[' + str(numberOfPage + 1) + ']/a')
                else:
                    bot.clickBtn('//*[@id="ContentPlaceHolder1_ctl03_divHO"]/div/div/table/tbody/tr/td[' + str(numberOfPage + 2) + ']/a')
                
            
        data[i] = dateAndClosingPrice
        if(len(data) == 100):
            break
        
    except NoSuchElementException:
        continue
#print(data)

Phần trăm hoàn tất:  11%|████▍                                   | 22/200 [27:35<3:33:00, 71.80s/it]

### In số lượng Stock đã cào được 

In [ ]:
#print(data['THI'])
print('Number of stock:', len(data))
print(data.keys())

Number of stock: 97
dict_keys(['CLL', 'HPX', 'HVN', 'SBT', 'CTS', 'VCI', 'HAR', 'TCT', 'PGI', 'DMC', 'MIG', 'PVD', 'DHA', 'VPH', 'VOS', 'SHI', 'VNM', 'AST', 'IDI', 'VCF', 'LGC', 'KDH', 'CCL', 'DBT', 'BFC', 'PDR', 'CTF', 'OPC', 'DGC', 'AGM', 'LAF', 'ROS', 'PAC', 'MCG', 'TMT', 'VJC', 'PC1', 'TMP', 'CRE', 'TVS', 'ACB', 'TGG', 'SMB', 'BRC', 'CEE', 'TNT', 'SGR', 'NHH', 'ITA', 'HRC', 'GTA', 'QCG', 'HOT', 'DTL', 'DIG', 'PIT', 'TDM', 'SII', 'SPM', 'PHC', 'VGC', 'HDG', 'FMC', 'TTE', 'TV2', 'DCL', 'SRF', 'EIB', 'VSH', 'LCG', 'KSB', 'GDT', 'HAX', 'IBC', 'NKG', 'VRE', 'HAG', 'GMC', 'SJF', 'SKG', 'ITD', 'NT2', 'DBC', 'KBC', 'D2D', 'KOS', 'POW', 'TLG', 'VDS', 'HID', 'HCD', 'PMG', 'GEX', 'HMC', 'SFC', 'DSN', 'TPC'])


## Xử lý dữ liệu và xuất file CSV

### Xử lý dữ liệu

In [ ]:
print(len(data))

97


In [ ]:
dataCopy = copy.deepcopy(data)
dataCopy = handleDuplicateData(dataCopy)
stockSymbolData = handleMissingDate(dataCopy)

### Xuất file CSV

In [ ]:


handleCSV = CSVHandler()
handleCSV.createDataFrame(stockSymbolData)
handleCSV.exportCSV("ClosingPrice.csv")